In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import copy
import warnings

warnings.filterwarnings('ignore')

In [2]:
scores_df = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/season_data.csv')
playoff_scores_df = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/playoff_data.csv')
real_elo = pd.read_csv('C:/Users/Gabriel/Documents/GitHub/MIE368_PROJECT/team_proj/data/nhl_elo.csv')

In [3]:
scores_df.head()

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season
0,2000-10-04,Colorado Avalanche,2.0,Dallas Stars,2.0,1,2001
1,2000-10-05,Ottawa Senators,4.0,Boston Bruins,4.0,1,2001
2,2000-10-05,Chicago Blackhawks,2.0,Buffalo Sabres,4.0,0,2001
3,2000-10-05,Detroit Red Wings,4.0,Calgary Flames,3.0,0,2001
4,2000-10-05,Vancouver Canucks,3.0,Philadelphia Flyers,6.0,0,2001


## Game Result Elo Adjustment

In [3]:
first_yr = scores_df[scores_df['Season'] == 2001]
Teams = first_yr['Home'].unique().tolist()

team_changes = {
    'Atlanta Thrashers': 'Winnipeg Jets',
    'Phoenix Coyotes': 'Arizona Coyotes',
    'Mighty Ducks of Anaheim': 'Anaheim Ducks'
    # Add more mappings as needed
}

K = 6

In [4]:
def calculate_prob_winning(home_team, away_team, elo_ratings):
    
    Elo_diff_home = elo_ratings[home_team]-elo_ratings[away_team] + 26  #add extra for home-ice advantage   
    #Elo_diff_away = elo_ratings[away_team] - elo_ratings[home_team]
    prob_win_home = 1/(10**(-1*Elo_diff_home/400)+1)
    #prob_win_away = 1/(10**(-1*Elo_diff_away/400)+1)
    prob_win_away = 1 - prob_win_home
    
    return prob_win_home, prob_win_away, Elo_diff_home

In [5]:
def margin_of_victory(home_goals, away_goals):
    
    mov = abs(home_goals - away_goals)
    mov_mult = 0.6686*np.log(mov)+0.8048
    
    return mov_mult

In [6]:
# Function to update ELO ratings after a game -- Could add input for elo_ratings dictionary 
def update_game_elo(home_team, away_team, home_goals, away_goals, season, df, idx, elo_ratings):
    
    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)
    
    #print('Home team is ' + str(home_team))
    #print('Away team is ' + str(away_team))

    if home_team not in elo_ratings:
        # New teams added past a certain year start with a different ELO rating
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)
    
    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
     # Update Pregame Favorite Multiplier
    if home_goals > away_goals:
        home_win = 1
        #away_win = 0
        winner_elo_diff = Elo_diff_home
    elif home_goals < away_goals:
        home_win = 0
        #away_win = 1
        winner_elo_diff = -1*Elo_diff_home
    else:
        return df, elo_ratings
    
    pre_g_fav_h = home_win - home_prob  
    #pre_g_fav_a = away_win - away_prob  
    
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
     # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(home_goals, away_goals)
    #print('home goals:' + str(home_goals))
    #print('away goals:' + str(away_goals))
    #print('mult is: ' + str(mov_multiplier))
    
     # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
    #elo_shift_a = K * mov_multiplier * pre_g_fav_a
    #print('elo shift is: ' + str(elo_shift_h))
    
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    return df, elo_ratings

## End of Season Elo Adjustments

In [7]:
def elo_adjuster(df):
    
    # Initialize variables to store the final Elo ratings for the season
    final_elo_ratings = {}
    elo_ratings = {}
    season_weight = 0.8
    average_weight = 0.2
    current_season = None

    # Iterate over seasons and games
    for idx, row in df.iterrows():
        if current_season is None:
            current_season = row['Season']
    
        # Check if the season in the current row is different from the current season
        if row['Season'] != current_season:
            # Save the previous season's Elo ratings and calculate the starting Elo for the new season
            final_elo_ratings[current_season] = dict(elo_ratings)
            elo_ratings = {}  # Reset Elo ratings for the new season

            for team in final_elo_ratings[current_season]:
                prev_season_elo = final_elo_ratings[current_season][team]
                league_avg_elo = sum(final_elo_ratings[current_season].values()) / len(final_elo_ratings[current_season])
                starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
                elo_ratings[team] = round(starting_elo, 2)

            current_season = row['Season']
            
        # Update Elo ratings for the game
        df, elo_ratings = update_game_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], row['Season'], df, idx, elo_ratings)

    # Calculate the final Elo ratings for the last season
    final_elo_ratings[current_season] = dict(elo_ratings)

    return df, elo_ratings, final_elo_ratings

In [8]:
seasons, elo, final_season_elos = elo_adjuster(scores_df)

In [21]:
elo

{'Dallas Stars': 1550.940150407058,
 'Colorado Avalanche': 1584.5444245615597,
 'Boston Bruins': 1646.0070614177143,
 'Ottawa Senators': 1476.0714294103982,
 'Buffalo Sabres': 1483.3405044471829,
 'Chicago Blackhawks': 1390.3846749520826,
 'Calgary Flames': 1517.6920572246158,
 'Detroit Red Wings': 1438.9591928296538,
 'Philadelphia Flyers': 1417.745218787118,
 'Vancouver Canucks': 1484.9400838907918,
 'Arizona Coyotes': 1401.4428712249032,
 'St. Louis Blues': 1486.1607904424352,
 'Edmonton Oilers': 1582.4085250119847,
 'Florida Panthers': 1541.7356119720528,
 'Anaheim Ducks': 1365.7461410120907,
 'Minnesota Wild': 1547.1258135936355,
 'New Jersey Devils': 1529.031110424827,
 'Montreal Canadiens': 1405.9111077584168,
 'Pittsburgh Penguins': 1507.3256629722844,
 'Nashville Predators': 1506.5670113535894,
 'San Jose Sharks': 1382.9101359293545,
 'Tampa Bay Lightning': 1542.7115664597802,
 'New York Islanders': 1513.7599713609845,
 'Washington Capitals': 1477.297610220473,
 'Los Angeles K

### Testing Bounds of Elo

In [11]:
for i in final_season_elos:
    print(f'Max of year {i} is {max(final_season_elos[i].values())}')
    print(f'Min of year {i} is {min(final_season_elos[i].values())}')

Max of year 2001 is 1592.1355800566328
Min of year 2001 is 1407.17797613946
Max of year 2002 is 1594.5595423047896
Min of year 2002 is 1375.969112961436
Max of year 2003 is 1604.4834324889157
Min of year 2003 is 1403.313756619737
Max of year 2004 is 1600.703277602256
Min of year 2004 is 1381.994928786802
Max of year 2006 is 1621.8315238665518
Min of year 2006 is 1385.2106980872195
Max of year 2007 is 1585.809806922114
Min of year 2007 is 1410.9965387341117
Max of year 2008 is 1592.415920121985
Min of year 2008 is 1439.730361842258
Max of year 2009 is 1573.4170823439883
Min of year 2009 is 1398.4869054830074
Max of year 2010 is 1584.8075179050084
Min of year 2010 is 1423.9071652779273
Max of year 2011 is 1591.4007466678427
Min of year 2011 is 1405.5966526380455
Max of year 2012 is 1577.0127514839012
Min of year 2012 is 1431.492209051298
Max of year 2013 is 1591.2735552272406
Min of year 2013 is 1419.8150830163438
Max of year 2014 is 1588.4750973383004
Min of year 2014 is 1394.2163417588

In [16]:
season_weight = 0.8
average_weight = 0.2
max_min = {}
min_ls = []
max_ls = []

for i in final_season_elos:
    start_elo = {}
    for team in final_season_elos[i]:
        prev_season_elo = final_season_elos[i][team]
        league_avg_elo = sum(final_season_elos[i].values()) / len(final_season_elos[i])
        starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
        start_elo[team] = round(starting_elo, 2)
    max_val = max(start_elo.values())
    min_val = min(start_elo.values())
    max_ls.append(max_val)
    min_ls.append(min_val)
    max_min[str(i)+' max'] = max_val
    max_min[str(i)+' min'] = min_val

In [20]:
print(f'The absolute max is {max(max_ls)}')
print(f'The absolute min is {min(min_ls)}')
print(f'The average max is {sum(max_ls) / len(max_ls)}')
print(f'The average min is {sum(min_ls) / len(min_ls)}')

The absolute max is 1616.81
The absolute min is 1388.25
The average max is 1574.9254545454548
The average min is 1419.6604545454543


In [17]:
max_min

{'2001 max': 1573.71,
 '2001 min': 1425.74,
 '2002 max': 1575.65,
 '2002 min': 1400.78,
 '2003 max': 1583.59,
 '2003 min': 1422.65,
 '2004 max': 1580.56,
 '2004 min': 1405.6,
 '2006 max': 1597.47,
 '2006 min': 1408.17,
 '2007 max': 1568.65,
 '2007 min': 1428.8,
 '2008 max': 1573.93,
 '2008 min': 1451.78,
 '2009 max': 1558.73,
 '2009 min': 1418.79,
 '2010 max': 1567.85,
 '2010 min': 1439.13,
 '2011 max': 1573.12,
 '2011 min': 1424.48,
 '2012 max': 1561.61,
 '2012 min': 1445.19,
 '2013 max': 1573.02,
 '2013 min': 1435.85,
 '2014 max': 1570.78,
 '2014 min': 1415.37,
 '2015 max': 1563.92,
 '2015 min': 1389.55,
 '2016 max': 1558.31,
 '2016 min': 1435.69,
 '2017 max': 1578.44,
 '2017 min': 1408.81,
 '2018 max': 1556.13,
 '2018 min': 1422.12,
 '2019 max': 1598.84,
 '2019 min': 1428.51,
 '2020 max': 1574.84,
 '2020 min': 1388.25,
 '2021 max': 1565.67,
 '2021 min': 1425.16,
 '2022 max': 1576.73,
 '2022 min': 1419.51,
 '2023 max': 1616.81,
 '2023 min': 1392.6}

## Building Elo Forecaster Function

In [48]:
#np.random.seed(0)
def forecast_game_elo(home_team, away_team, season, df, idx, elo_ratings, simulation_wins):

    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)

    if home_team not in elo_ratings:
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)

    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
    home_win = np.random.choice([0, 1], p=[away_prob, home_prob])

    # Calculate base scores for the home and away team
    score_home = 2.845905 + 0.003584*Elo_diff_home
    score_away = 2.845905 + 0.003584*(-1)*Elo_diff_home

    conditions_satisfied = 0 

    while conditions_satisfied == 0:
        
        # poisson distribution to shift scores
        poisson_score_home = np.random.poisson(max(0, score_home))
        poisson_score_away = np.random.poisson(max(0, score_away))

        home_goal_diff = poisson_score_home - poisson_score_away
            
        # does it satsify conditions?
        if home_win == 1:
            if home_goal_diff > 0:
                conditions_satisfied = 1
        
        if home_win == 0:
            if home_goal_diff < 0: 
                conditions_satisfied = 1

    #------------------------------------------------------------------------#
    OT_choice = 0
    
    if abs(home_goal_diff) == 1:
        # Decide if the game goes to OT
        OT_choice = np.random.choice(a= [1, 0], p= [0.505, 0.495])
            
    if home_win == 1:
        winner_elo_diff = Elo_diff_home
        simulation_wins[home_team] += 2

        if OT_choice == 1:
            simulation_wins[away_team] += 1
        
    if home_win == 0:
        winner_elo_diff = -1*Elo_diff_home
        simulation_wins[away_team] += 2

        if OT_choice == 1:
            simulation_wins[home_team] += 1

    pre_g_fav_h = home_win - home_prob  
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
    # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(poisson_score_home, poisson_score_away)

    # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
  
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    df.at[idx, 'G_Home'] = poisson_score_home
    df.at[idx, 'G_Vis'] = poisson_score_away
    df.at[idx, 'OT Ind'] = OT_choice

    #--------------------------------------------------------------------------#

    return df, elo_ratings, simulation_wins

In [49]:
# Iterate over seasons and games
def season_forecaster(df, starting_elo):
        
    # Create a dictionary to track point totals
    simulation_pts = {team: 0 for team in df['Home'].unique()}

    for idx, row in df.iterrows():
        df, starting_elo, simulation_pts = forecast_game_elo(row['Home'], row['Visitor'], row['Season'], df, idx, starting_elo, simulation_pts)  
        
    return df, starting_elo, simulation_pts

In [50]:
def playoff_tracker(simulation_pts, playoffs_made):

    conference_mapping = {
        'East': ['Florida Panthers', 'Toronto Maple Leafs', 'Tampa Bay Lightning', 'Boston Bruins', 'Buffalo Sabres', 'Detroit Red Wings',
                 'Ottawa Senators', 'Montreal Canadiens', 'Carolina Hurricanes', 'New York Rangers', 'Pittsburgh Penguins', 
                 'Washington Capitals', 'New York Islanders', 'Columbus Blue Jackets', 'New Jersey Devils', 'Philadelphia Flyers'],
        'West': ['Colorado Avalanche', 'Minnesota Wild', 'St. Louis Blues', 'Dallas Stars', 'Nashville Predators', 'Winnipeg Jets',
                 'Chicago Blackhawks', 'Arizona Coyotes', 'Calgary Flames', 'Edmonton Oilers', 'Los Angeles Kings', 'Vegas Golden Knights',
                 'Vancouver Canucks', 'San Jose Sharks', 'Anaheim Ducks', 'Seattle Kraken']
    }
    
    # Separate teams into East and West lists
    for conference, teams in conference_mapping.items():
        valid_teams = [i for i in teams if i in simulation_pts]
        sorted_teams = sorted(valid_teams, key=lambda x: simulation_pts[x], reverse=True)

        for team in sorted_teams[:8]:
            playoffs_made[team] += 1

    return playoffs_made

In [54]:
np.random.seed(0)

df2 = copy.deepcopy(scores_df[scores_df['Season'] == 2022])
season_2022 = df2.copy() 
num_simulations = 500
season_weight = 0.7
average_weight = 0.3

# Create a dictionary to track overall standings
standings = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}

# Create a dictionary to track playoff appearances
playoffs_counter = {team: 0 for team in df2['Home'].unique()}

for simulation in range(num_simulations):
    elo_end = copy.deepcopy(final_season_elos[2021])  # Creating a deep copy of final_season_elos[2021]

    elo_ratings = {}  # Reset Elo ratings for the new season

    for team in elo_end:
        prev_season_elo = elo_end[team]
        league_avg_elo = sum(elo_end.values()) / len(elo_end)
        starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
        elo_ratings[team] = round(starting_elo, 2)

    season_2022, elo_ratings, simulation_pts = season_forecaster(season_2022, elo_ratings)
    playoffs_counter = playoff_tracker(simulation_pts, playoffs_counter)

    for team, pts in simulation_pts.items():
        standings[team][simulation] = pts


for team, pts in standings.items():
    #print(pts)
    mean_pts = np.mean(pts)
    std_deviation = np.std(pts)
    print(f"{team}: Mean point total = {mean_pts}, Standard Deviation = {std_deviation}")

Tampa Bay Lightning: Mean point total = 93.934, Standard Deviation = 13.078747799388134
Vegas Golden Knights: Mean point total = 98.812, Standard Deviation = 12.9284436805054
Anaheim Ducks: Mean point total = 73.044, Standard Deviation = 11.95533621442743
Colorado Avalanche: Mean point total = 98.446, Standard Deviation = 12.186840607803157
Edmonton Oilers: Mean point total = 90.728, Standard Deviation = 12.910384037665185
Toronto Maple Leafs: Mean point total = 92.726, Standard Deviation = 12.9020511547583
Washington Capitals: Mean point total = 93.496, Standard Deviation = 12.789760904723746
Buffalo Sabres: Mean point total = 71.726, Standard Deviation = 12.055659417883371
Carolina Hurricanes: Mean point total = 96.434, Standard Deviation = 12.415218242141375
Columbus Blue Jackets: Mean point total = 74.872, Standard Deviation = 12.639130349830246
Detroit Red Wings: Mean point total = 71.688, Standard Deviation = 12.047848604626472
Florida Panthers: Mean point total = 93.378, Standar

### Finding Runtime Issues

In [19]:
import cProfile

def main():
    season_2022 = copy.deepcopy(scores_df[scores_df['Season'] == 2022])
    num_simulations = 500
    season_weight = 0.7
    average_weight = 0.3

    # Create a dictionary to track overall standings
    standings = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}

    # Create a dictionary to track playoff appearances
    playoffs_counter = {team: 0 for team in df2['Home'].unique()}

    for simulation in range(num_simulations):
        elo_end = copy.deepcopy(final_season_elos[2021])  # Creating a deep copy of final_season_elos[2021]

        elo_ratings = {}  # Reset Elo ratings for the new season

        for team in elo_end:
            prev_season_elo = elo_end[team]
            league_avg_elo = sum(elo_end.values()) / len(elo_end)
            starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
            elo_ratings[team] = round(starting_elo, 2)

        season_2022, elo_ratings, simulation_pts = season_forecaster(season_2022, elo_ratings)
        playoffs_counter = playoff_tracker(simulation_pts, playoffs_counter)

        for team, pts in simulation_pts.items():
            standings[team][simulation] = pts


    for team, pts in standings.items():
        #print(pts)
        mean_pts = np.mean(pts)
        std_deviation = np.std(pts)
        print(f"{team}: Mean point total = {mean_pts}, Standard Deviation = {std_deviation}")

if __name__ == "__main__":
    # Run the profiler
    cProfile.run('main()', sort='cumulative')

         2082 function calls (1890 primitive calls) in 0.005 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.005    0.005 3134879366.py:3(main)
        1    0.000    0.000    0.002    0.002 4235467807.py:2(season_forecaster)
        1    0.000    0.000    0.001    0.001 indexing.py:2481(__getitem__)
        1    0.000    0.000    0.001    0.001 indexing.py:2431(__getitem__)
        3    0.000    0.000    0.001    0.000 base.py:3762(get_loc)
        1    0.000    0.000    0.001    0.001 frame.py:3982(_get_value)
        5    0.000    0.000    0.001    0.000 frame.py:3853(__getitem__)
     64/2    0.000    0.000    0.001    0.000 copy.py:128(deepcopy)
        3    0.001    0.000    0.001    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine'

KeyError: 0

### Forecast Results

In [129]:
len(season_2022[season_2022['OT Ind'] == 1])/len(season_2022)

0.22332317073170732

In [130]:
mean_pts = pd.DataFrame(columns = ['PTS']).rename_axis('Team')

for team, pts in standings.items():
    mean_pts.loc[team] = [np.mean(pts)]



In [131]:
mean_pts.sort_values(by='PTS', ascending=False)

,PTS
Team,
Colorado Avalanche,98.33
Vegas Golden Knights,96.86
Boston Bruins,95.88
Carolina Hurricanes,95.05
Tampa Bay Lightning,94.29
Pittsburgh Penguins,94.23
Florida Panthers,94.15
Toronto Maple Leafs,93.91
Washington Capitals,92.83


In [132]:
print("Elo ratings at the end of the simulation")

elo_df = pd.DataFrame(columns = ['Elo']).rename_axis('Team')

for team, elo in elo_ratings.items():
    elo_df.loc[team] = [elo]

elo_df.sort_values(by='Elo', ascending=False)

Elo ratings at the end of the simulation


,Elo
Team,
Washington Capitals,1599.556568
Tampa Bay Lightning,1574.939080
Pittsburgh Penguins,1567.182912
Carolina Hurricanes,1559.041005
New York Islanders,1547.307066
Arizona Coyotes,1532.789055
Vegas Golden Knights,1532.414202
Minnesota Wild,1532.308247
Montreal Canadiens,1529.608428


In [35]:
final_season_elos[2022]

{'Dallas Stars': 1502.973236337635,
 'Colorado Avalanche': 1592.581164736313,
 'Boston Bruins': 1560.547495955879,
 'Ottawa Senators': 1451.4854033719362,
 'Buffalo Sabres': 1442.484547886203,
 'Chicago Blackhawks': 1419.2678865381408,
 'Calgary Flames': 1562.9107481215176,
 'Detroit Red Wings': 1415.3240300648633,
 'Philadelphia Flyers': 1404.1010905547882,
 'Vancouver Canucks': 1504.8608226402444,
 'Arizona Coyotes': 1408.1316559496277,
 'St. Louis Blues': 1562.2001817243752,
 'Edmonton Oilers': 1552.2017787736845,
 'Florida Panthers': 1589.3969872086652,
 'Anaheim Ducks': 1423.582715781608,
 'Minnesota Wild': 1568.5231613538795,
 'New Jersey Devils': 1414.0095382202796,
 'Montreal Canadiens': 1401.1543457998162,
 'Pittsburgh Penguins': 1546.6893035642772,
 'Nashville Predators': 1516.6364426933594,
 'San Jose Sharks': 1424.0034968936147,
 'Tampa Bay Lightning': 1567.6614901775442,
 'New York Islanders': 1501.4992789850435,
 'Washington Capitals': 1533.356501163791,
 'Los Angeles Kin

In [34]:
print("Elo ratings at the end of the real prior season")

elo_df = pd.DataFrame(columns = ['Elo']).rename_axis('Team')

for team, elo in final_season_elos[2022].items():
    elo_df.loc[team] = [elo]

elo_df.sort_values(by='Elo', ascending=False)

Elo ratings at the end of the real prior season


,Elo
Team,
Colorado Avalanche,1592.581165
Florida Panthers,1589.396987
Carolina Hurricanes,1581.955223
Toronto Maple Leafs,1573.327152
Minnesota Wild,1568.523161
Tampa Bay Lightning,1567.661490
Calgary Flames,1562.910748
St. Louis Blues,1562.200182
Boston Bruins,1560.547496


In [140]:
print("Playoff Probabilities")

playoff_df = pd.DataFrame(columns = ['Playoff Prob']).rename_axis('Team')

playoffs_counter_normalized = {team: count / num_simulations for team, count in playoffs_counter.items()}

for team, playoff_prob in playoffs_counter_normalized.items():
    playoff_df.loc[team] = [playoff_prob]

playoff_df.sort_values(by='Playoff Prob', ascending=False)

Playoff Probabilities


,Playoff Prob
Team,
Vegas Golden Knights,0.86
Boston Bruins,0.81
Colorado Avalanche,0.78
Tampa Bay Lightning,0.75
Florida Panthers,0.75
Carolina Hurricanes,0.74
Pittsburgh Penguins,0.73
Toronto Maple Leafs,0.71
New York Islanders,0.70


In [13]:
season_2022

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season,EloDiffHome,EloDiffAway,WinProbHome,WinProbVis
23662,2021-10-12,Pittsburgh Penguins,1.0,Tampa Bay Lightning,3.0,0,2022,24.150000,-24.150000,0.534699,0.465301
23663,2021-10-12,Seattle Kraken,2.0,Vegas Golden Knights,1.0,0,2022,82.260000,-82.260000,0.616218,0.383782
23664,2021-10-13,Winnipeg Jets,7.0,Anaheim Ducks,1.0,0,2022,-36.470000,36.470000,0.447707,0.552293
23665,2021-10-13,Chicago Blackhawks,1.0,Colorado Avalanche,0.0,0,2022,100.820000,-100.820000,0.641152,0.358848
23666,2021-10-13,Vancouver Canucks,6.0,Edmonton Oilers,1.0,0,2022,72.510000,-72.510000,0.602861,0.397139
...,...,...,...,...,...,...,...,...,...,...,...
24969,2022-04-29,San Jose Sharks,2.0,Seattle Kraken,4.0,0,2022,75.722038,-75.722038,0.607280,0.392720
24970,2022-04-29,Vegas Golden Knights,1.0,St. Louis Blues,5.0,0,2022,13.081857,-13.081857,0.518817,0.481183
24971,2022-04-29,Boston Bruins,5.0,Toronto Maple Leafs,2.0,0,2022,-47.719030,47.719030,0.431755,0.568245
24972,2022-04-29,Calgary Flames,2.0,Winnipeg Jets,4.0,0,2022,116.315306,-116.315306,0.661406,0.338594
